# Carga de datos

In [ ]:
import pandas as pd
import numpy as np
import datetime 

In [ ]:
#Carga de datos con Embalses y rios
df=pd.read_csv('data_rios_embalses.csv',index_col=[0])
df.columns = map(lambda x: str(x).lower(), df.columns)
df['mes']=df['fecha'].str[:7]
df

In [ ]:
#Carga de datos climáticos
tiempo=pd.read_excel('tiempo.xlsx')
tiempo.columns = map(lambda x: str(x).lower(), tiempo.columns)
tiempo['provincia'] = tiempo['provincia'].str.upper()
tiempo['estación'] = tiempo['estación'].str.upper()
tiempo

In [ ]:

tiempo['idprovincia'] = tiempo['provincia'].apply(lambda x: 15 if x=='A CORUÑA' else (32 if x=='OURENSE' else (27 if x=='LUGO' else 36)))

In [ ]:
tiempo=tiempo.drop_duplicates()

In [ ]:
tiempo.to_csv('tiempo.csv')

In [ ]:
tiempo1=pd.read_csv('tiempo.csv',index_col=[0])
tiempo1

In [ ]:
tiempo1['mes']=tiempo1['fecha'].str[:7]

In [ ]:
tiempo1

In [ ]:
#Join Data
df_ret =pd.merge(df, tiempo1, on =['mes','idprovincia'], how ='inner')
df_ret

In [ ]:
df_ret[df_ret['id']==2008320968][['estación','latitud','longitud']]

# Función de distancias de latitudes y longitudes

In [ ]:
from math import radians, cos, sin, asin, sqrt 
def distance(lat1, lat2, lon1, lon2): 
      
    # radians which converts from degrees to radians. 
    lon1 = radians(lon1) 
    lon2 = radians(lon2) 
    lat1 = radians(lat1) 
    lat2 = radians(lat2) 
       
    # Haversine formula  
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  
    c = 2 * asin(sqrt(a))  
     
    # Radius of earth in kilometers. Use 3956 for miles 
    r = 6371
       
    # calculate the result 
    return(c * r) 

def dist_min(df1,column_lat1,column_lat2,column_lng1,column_lng2,index,name):
    
    #Add column containing distances 
    df1[name]=df1.apply(lambda x: distance(x[column_lat1], x[column_lat2], x[column_lng1], x[column_lng2]), axis=1)
    #Selecting the min distance per id
    df1_min = df1.groupby([index]).agg({name: 'min'})
    df2 = pd.merge(df1, df1_min, on = index, how ='inner')
    df = df2[(df2[name+'_x']==df2[name+'_y']) | df2[name+'_x'].isna()]
    return(df)

In [ ]:
from tqdm import tqdm
df_fin = dist_min(df_ret, 'lat', 'latitud', 'lng', 'longitud', 'id','dist_est')
df_fin

In [ ]:
df_fin[df_fin['id']==2008320968][['estación','latitud','longitud']]

In [ ]:
df_fin.groupby(['id']).agg({'superficie': 'count'}).sort_values(by = 'superficie')

In [ ]:
df_fin[df_fin['id']==2011322068][['dist_est_x','dist_est_y','estación','latitud','longitud','temp_media']]

In [ ]:
df_fin=df_fin.drop_duplicates()

In [ ]:
%%time
tiempo1[(tiempo1['mes']=='2008-07')&(tiempo1['idprovincia']==32)]

In [ ]:
df_fin.columns

## Regresión Logística

### Creación de datos

## añadir los ids por si hay mas de un incendio por mes y coordendas


In [ ]:
#inc1=pd.DataFrame({'mes': df_fin['mes'],'coord': list(zip(df_fin.lat, df_fin.lng)), 'lat': df_fin['lat'],'lng': df_fin['lng'],'temp_med':df_fin['temp_media'],'temp_mx_med': df_fin['temp_max_med'], 'temp_mn_med':df_fin['temp_min_med'], 'precipitacion':df_fin['prec_acu'],'hum_med':df_fin['hum_med'], 'vel_viento':df_fin['v_viento_med'], 'presion':df_fin['presion'], 'lluvia':df_fin['lluvia']})
#inc1['inc']=1
inc1=df_fin[['mes','idprovincia','lat','lng']]
inc1['coord']=list(zip(inc1.lat, inc1.lng))
inc1

In [ ]:
incendios=inc1[['coord','lat','lng','idprovincia']]
incendios=incendios.drop_duplicates()

In [ ]:
incendios=incendios.reset_index()

In [ ]:
incendios

In [ ]:
inc1=inc1.reset_index()
inc1

In [ ]:
inc=pd.DataFrame()

In [ ]:
inc1['mes'].unique()

In [ ]:
%%time
for x in inc1['mes'].unique():
    incendiosx=incendios
    incendiosx['mes']=x
    incendiosx['lume']=1
    inc=inc.append(incendiosx)
    for y in incendios['index']:
        if inc1[(inc1['mes']==x) & (inc1['index']==y)].empty: 
            inc.lume[(inc['mes']==x)&(inc['index']==y)]=0
        else: None


In [ ]:
inc.to_csv('logistic_df.csv')

In [ ]:
inc[(inc['mes']=='2001-02') & (inc1['idprovincia']==15)]


In [ ]:
inc[inc['lume']==1].shape,inc[inc['lume']==0].shape

In [ ]:
inc

In [ ]:
inc1[(inc1['mes']=='2001-02') & (inc1['idprovincia']==15)]

In [ ]:
df_fin =pd.merge(inc, tiempo1, on =['mes','idprovincia'], how ='inner')

In [ ]:
df_fin = dist_min(df_fin, 'lat', 'latitud', 'lng', 'longitud', 'index','dist_est')

In [ ]:
df_fin.columns

## Modelo

In [ ]:
z=df_fin[['temp_media',
       'temp_max_med', 'temp_min_med', 'prec_acu', 'hum_med', 
        'v_viento_med', 'lluvia', 'helada','lume']]
z=z.dropna()

In [ ]:
X=z[['temp_media',
       'temp_max_med', 'temp_min_med', 'prec_acu', 'hum_med', 
        'v_viento_med', 'lluvia', 'helada']]
y=z['lume']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
clf=LogisticRegression()

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
clf.predict(X_test)

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fp,tp,_ =roc_curve(y_test, clf.predict_proba(X_test)[:,1])
plt.plot(fp,tp)

In [ ]:
y_test

In [ ]:
clf.predict_proba(X_test)[:,1]

In [ ]:
X

### Libreria folium para plotear

### Serie temporal: modelo anova para correlacion entre variables

###### Sé que esto lo sabías, no te enfades bebé, te amo

In [ ]:
df_fin.shape